# eidein

Interactive tool for dimensionality reduction of astronomical data and their visualisation.

In [1]:
%matplotlib widget

from astropy.modeling import models, fitting
import h5py
import ipywidgets
import matplotlib.pyplot as plt
import numpy as np

from eidein import Eidein

## Data

In [2]:
SIZE = 2048

In [3]:
# TODO should I take only those with high S/N?
with h5py.File("data/dr16q_superset.hdf5", "r") as datafile:
    print(datafile.keys())
    n = datafile["id"].shape[0]
    mi = datafile["mutual_information"][:]
    idx = np.sort(np.argsort(mi)[-2048:])
    mi = mi[idx]
    ids = datafile["id"][idx]
    X = datafile["X"][idx]
    z_pred = datafile["z_pred"][idx]
ids.dtype, X.dtype, mi.dtype, z_pred.dtype

<KeysViewHDF5 ['X', 'entropy', 'entropy_std', 'id', 'mutual_information', 'pipe_corr_10k', 'sn_median_all', 'source_z', 'variation_ratio', 'z', 'z_10k', 'z_pca', 'z_pipe', 'z_pred', 'z_pred_std', 'z_qn', 'z_vi', 'zs_pred']>


(dtype('int32'), dtype('float32'), dtype('float64'), dtype('float32'))

In [4]:
def id2filename(plate, mjd, fiberid):
    return "spec-{:04d}-{}-{:04d}.fits".format(plate, mjd, fiberid)

vec_id2filename = np.vectorize(id2filename)
identifiers = vec_id2filename(ids[:, 0], ids[:, 1], ids[:, 2])

In [5]:
fig, ax = plt.subplots()
ax.hist(mi)
ax.set_xlabel("Mutual Information")
ax.set_ylabel("Count")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Count')

In [6]:
LOGLAMMIN, LOGLAMMAX = 3.5832, 3.9583
N_FEATURES = 3752
LAM = np.power(10, np.linspace(LOGLAMMIN, LOGLAMMAX, N_FEATURES))
LINES = [
    (1033.82, "O VI"),
    (1215.24, "Lyα"),
    (1240.81, "N V"),
    (1399.8, "Si IV + O IV"),
    (1549.48, "C IV"),
    (1908.734, "C III"),
    (2326.0, "C II"),
    (2799.117, "Mg II"),
    (3727.092, "O II"),
    (4102.89, "HΔ"),
    (4341.68, "Hγ"),
    (4862.68, "Hβ"),
    (4960.295, "O III"),
    (5008.240, "O III"),
    (6564.61, "Hα")]
ARROWPROPS = {"arrowstyle": "-|>", "facecolor": "black"}

def z2lam_emit(z, lam_obsv):
    return lam_obsv / (1 + z)

def plot_spectrum(ax, filename, flux, y, label, lam=LAM):
    label_str = "{}\n$\hat{{z}} = {:.2f}$\n$z = {:.2f}$ (shown)".format(
        filename, y, label)
    
    lam_emit = [z2lam_emit(label, l) for l in lam]
    ax.plot(lam_emit, flux, label=label_str)
    ax.legend()
    ax.set_xlabel("Rest Frame Wavelength [Å]")
    ax.set_ylabel("Flux [10$^{-17}$ erg cm$^{-2}$ s$^{-1}$ Å$^{-1}$]")
    
    # plot spectral lines
    for line, name in LINES:
        ax.annotate(
            name,
            xy=(line, 0), xytext=(line, -2),
            arrowprops=ARROWPROPS,
            horizontalalignment='center')

In [8]:
rnd_i = np.random.randint(SIZE)
fig, ax = plt.subplots()
plot_spectrum(ax, identifiers[rnd_i], X[rnd_i], z_pred[rnd_i], z_pred[rnd_i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
labels = ipywidgets.FloatText(description="Redshift:", step=0.01)
eidein = Eidein(identifiers, X, mi, plot_spectrum, labels)
display(eidein)

Eidein(children=(Tab(children=(interactive(children=(Checkbox(value=False, description='whiten'), Dropdown(des…

In [10]:
eidein.labelled

{'spec-4457-55858-0578.fits': 4.27}